<a href="https://colab.research.google.com/github/torresmateo/penguin-tf-workshop/blob/master/D4_1_Series_temporales.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Explorando las Series Temporales

En este notebook vamos a ver como generar series temporales, para entender conceptos como la tendencia, patrones de temporada, autocorrelación, y ruido.

Primero que nada, se incluyen las bibliotecas necesarias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

se provee una función para poder visualizar las series que vayamos creando

In [ ]:
def ver_serie(tiempo, valores):
    plt.figure(figsize=(10, 8))
    plt.plot(tiempo, valores)
    plt.xlabel("Tiempo")
    plt.ylabel("Valor")
    plt.grid(True)
    plt.show()

 # Tendencia

 Es la "pendiente" de la serie temporal. Puede ser creciente o decreciente. 

In [ ]:
def tendencia(tiempo, pendiente=0):
    return tiempo * pendiente

In [ ]:
tiempo = np.arange(5 * 365)
serie = tendencia(tiempo, 0.1)
ver_serie(tiempo, serie)

# Patrón de estacionalidad

Es una repetición periódica de valores en la serie, si no existe una tendencia (pendiente 0) y hay solamente un patrón de temporada, el período es lo único que perturba a los valores de la serie.

In [ ]:
def patron_estacion(tiempo_estacion):
    return np.where(tiempo_estacion < 0.4, np.sin(tiempo_estacion * 4 * np.pi ), 0.5*np.sin(tiempo_estacion * 5 * np.pi)**2 + np.cos(tiempo_estacion * 5)**2)

def estacionalidad(tiempo, periodo, amplitud = 1, fase = 0):
    tiempo_estacion = ((tiempo + fase) % periodo) / periodo
    return amplitud * patron_estacion(tiempo_estacion)

In [ ]:
serie = estacionalidad(tiempo, periodo=365, amplitud=20)
ver_serie(tiempo, serie)

# Una serie con tendencia y estacionalidad

In [ ]:
serie = 10 + tendencia(tiempo, 0.1) + estacionalidad(tiempo, periodo=365, amplitud=40)
ver_serie(tiempo, serie)

# Ruido

El ruido puede venir de interacciones de los valores medidos con otros fenómenos, o de fallas en el registro de los datos.

In [ ]:
def ruido(tiempo, nivel=1):
    return np.random.randn(len(tiempo)) * nivel

In [ ]:
serie_ruidosa = serie + ruido(tiempo, nivel=10)
ver_serie(tiempo, serie_ruidosa)

si el nivel de ruido sobrepasa por mucho a la varianza natural de nuestra serie, se pierde la mayoría del patrón que podemos detectar

In [ ]:
serie_ruidosa = serie + ruido(tiempo, nivel=80)
ver_serie(tiempo, serie_ruidosa)

# Autocorrelación

La autocorrelación en datos temporales significa que la señal correlaciona consigo misma.

In [ ]:
def autocorr_positiva(tiempo, amplitud):
    ar = np.random.randn(len(tiempo) + 80)
    ar = np.arange(len(tiempo) + 80)
    for step in range(80, len(tiempo) + 80):
        ar[step] += 0.6 * ar[step - 80] - 0.9 * ar[step - 20]
    return ar[80:] * amplitud

def autocorr_negativa(tiempo, amplitud):
    ar = np.random.randn(len(tiempo) + 80)
    ar = np.arange(len(tiempo) + 80)
    for step in range(80, len(tiempo) + 80):
        ar[step] -= 0.6 * ar[step - 80] - 0.9 * ar[step - 20]
    return ar[80:] * amplitud

serie = autocorr_positiva(tiempo, 10)
ver_serie(tiempo[:200], serie[:200])

# Serie completa



In [ ]:
serie = autocorr_positiva(tiempo, 10) + estacionalidad(tiempo, periodo=50, amplitud=150) + tendencia(tiempo, 2) + ruido(tiempo)
ver_serie(tiempo[:300], serie[:300])

In [ ]:
serie = autocorr_negativa(tiempo, 10) + estacionalidad(tiempo, periodo=50, amplitud=150) + tendencia(tiempo, 2) + ruido(tiempo)
ver_serie(tiempo, serie)

In [ ]:
serie = autocorr_positiva(tiempo, 10) + estacionalidad(tiempo, periodo=50, amplitud=150) + tendencia(tiempo, 2) + ruido(tiempo)
serie[300:] = autocorr_negativa(tiempo[300:], 10) + estacionalidad(tiempo[300:], periodo=20, amplitud=150) + tendencia(tiempo[300:], -2) + ruido(tiempo[300:])

ver_serie(tiempo[:600], serie[:600])

# Créditos 

Este notebook está inspirado en contenido del curso [TensorFlow in Practice](https://www.deeplearning.ai/tensorflow-in-practice/).